In [1]:
import pickle
from pathlib import Path

import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import configuration as config
from cardiac_electrophysiology.ulac import construction, workflow

In [2]:
patient_id = "01"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = {"MV": 0, "LAA": 1, "LIPV": 2, "LSPV": 3, "RIPV": 4, "RSPV": 5}

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [3]:
ulac_construction_settings = construction.ULACConstructorSettings(
    mesh=mesh,
    feature_tags=anatomical_tags,
    path_config=config.path_configs,
    marker_config=config.marker_configs,
    parameterization_config=config.parameterization_configs,
    uac_config=config.uac_configs,
    submesh_config=config.submesh_configs,
    segmentation_workflow=workflow.workflow,
)
ulac_constructor = construction.ULACConstructor(ulac_construction_settings)
ulac_constructor.construct_segmentation()
ulac_constructor.construct_uacs()

Starting Segmentation


Step 1/10: boundary_extraction
------------------------------
Extracting Feature: ('LIPV', 'inner')
Extracting Feature: ('LIPV', 'outer')
Extracting Feature: ('LSPV', 'inner')
Extracting Feature: ('LSPV', 'outer')
Extracting Feature: ('RSPV', 'inner')
Extracting Feature: ('RSPV', 'outer')
Extracting Feature: ('RIPV', 'inner')
Extracting Feature: ('RIPV', 'outer')
Extracting Feature: ('LAA',)
Extracting Feature: ('MV',)
 
Step 2/10: path_construction_roof_anchors
-----------------------------------------
Constructing Shortest Path: ('roof', 'LIPV_LSPV')
Constructing Shortest Path: ('roof', 'LSPV_RSPV')
Constructing Shortest Path: ('roof', 'RSPV_RIPV')
Constructing Shortest Path: ('roof', 'RIPV_LIPV')
Constructing Shortest Path: ('anchor', 'LIPV_LAA')
Constructing Shortest Path: ('anchor', 'LAA_MV')
Constructing Shortest Path: ('anchor', 'LSPV_MV')
Constructing Shortest Path: ('anchor', 'RSPV_MV')
Constructing Shortest Path: ('anchor', 'RIPV_MV')
 
Step 3/10: mark

In [4]:
marker_data = ulac_constructor.data.marker_data
uac_path_data = ulac_constructor.data.uac_path_data
marker_key_sequences = list(construction.nested_dict_keys(marker_data))
path_key_sequences = list(construction.nested_dict_keys(uac_path_data))

plotter = pv.Plotter(window_size=[700, 500])
plotter.add_mesh(mesh, style="surface", show_edges=True, color="grey")

for key_sequence in marker_key_sequences:
    marker = construction.get_dict_entry(key_sequence, marker_data)
    if marker is None:
        continue
    marker_mesh = pv.PolyData(mesh.points[marker])
    plotter.add_mesh(marker_mesh, color="red", point_size=15, render_points_as_spheres=True)
for key_sequence in path_key_sequences:
    path = construction.get_dict_entry(key_sequence, uac_path_data)
    if path is None:
        continue
    path_mesh = pv.PolyData(mesh.points[path.inds])
    path_mesh.point_data["relative_lengths"] = path.relative_lengths
    plotter.add_mesh(
        path_mesh, point_size=10, scalars="relative_lengths", render_points_as_spheres=True
    )

plotter.show()

Widget(value='<iframe src="http://localhost:44491/index.html?ui=P_0x7fd45fbd30e0_0&reconnect=auto" class="pyvi…

In [5]:
uac_submesh_data = ulac_constructor.data.uac_submesh_data
uac_submesh_key_sequences = list(construction.nested_dict_keys(uac_submesh_data))

plotter = pv.Plotter()
for key_sequence in uac_submesh_key_sequences:
    uac_submesh = construction.get_dict_entry(key_sequence, uac_submesh_data)
    if uac_submesh is None:
        continue
    coordinates = np.hstack(
        (
            uac_submesh.alpha[:, None],
            uac_submesh.beta[:, None],
            np.zeros((uac_submesh.alpha.shape[0], 1)),
        )
    )
    submesh = pv.PolyData.from_regular_faces(coordinates, uac_submesh.connectivity)
    plotter.add_mesh(submesh, style="wireframe", color="grey")
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:44491/index.html?ui=P_0x7fd45fdfd6d0_1&reconnect=auto" class="pyvi…